In [95]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data_file = "./KOBIS_개봉일람_2023-03-20.csv"
df = pd.read_csv(data_file, encoding='UTF-8',index_col = "순번")

In [477]:
# row 생략 없이 출력
pd.set_option('display.max_rows', 200)
# col 생략 없이 출력
pd.set_option('display.max_columns', 200)

In [96]:
df = df.drop(['제작사', '수입사'], axis=1)
df['감독'].fillna('기타', inplace=True)
df['장르'].fillna('기타', inplace=True)
df['배급사'].fillna('기타', inplace=True)
df['영화형태'].fillna('기타', inplace=True)

for i in df['개봉일'][df['개봉일'].isnull()].index:
    df = df.drop(i)

for i in df['전국 매출액'][df['전국 매출액'].isnull()].index:
    df = df.drop(i)

for i in df['등급'][df['등급'].isnull()].index:
    df = df.drop(i)

In [97]:
# 다수의 감독 데이터 변경
for idx, row in df.iterrows():
    if ',' in row['감독']:
        directors = row['감독'].split(',')
        # 양 끝 공백 제거 후 정렬
        directors.sort(key=lambda x : x.strip())
        df['감독'][idx] = directors[0].strip()

In [98]:
for idx, row in df.iterrows():
    if ',' in row['배급사']:
        distributor = row['배급사'].split(',')
        # 양 끝 공백 제거 후 변경
        df['배급사'][idx] = distributor[0].strip()

In [99]:
df.drop('영화유형', axis=1, inplace=True)

In [100]:
for idx, row in df.iterrows():
    if ',' in row['등급']:
        ratings = row['등급'].split(',')
        # 전체, 12, 15, 청소년관람불가 순
        if '전체' in ratings:
            df['등급'][idx] = '전체관람가'
        elif '12' in ratings:
            df['등급'][idx] = '12세관람가'
        elif '15' in ratings:
            df['등급'][idx] = '15세관람가'
        else:
            df['등급'][idx] = '청소년관람불가'
    elif df['등급'][idx] == '18세관람가' or df['등급'][idx] == '제한상영가': 
        df['등급'][idx] = '청소년관람불가'
    else:
        df['등급'][idx] = df['등급'][idx].replace('이상', '')

In [101]:
df['전국 매출액'] = df['전국 매출액'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 매출액'] = df['전국 매출액'].astype('int64')

df['전국스크린수'] = df['전국스크린수'].str.replace(pat=r',', repl=r'', regex=True)
df['전국스크린수'] = df['전국스크린수'].astype('int64')

df['전국 관객수'] = df['전국 관객수'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 관객수'] = df['전국 관객수'].astype('int64')

In [102]:
movie_resize = df[(df['전국 관객수'] > 1000) & (df['전국스크린수'] > 50)]
movie_resize.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4145 entries, 1 to 7676
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화명     4145 non-null   object
 1   감독      4145 non-null   object
 2   배급사     4145 non-null   object
 3   개봉일     4145 non-null   object
 4   영화형태    4145 non-null   object
 5   국적      4145 non-null   object
 6   전국스크린수  4145 non-null   int64 
 7   전국 매출액  4145 non-null   int64 
 8   전국 관객수  4145 non-null   int64 
 9   장르      4145 non-null   object
 10  등급      4145 non-null   object
 11  영화구분    4145 non-null   object
dtypes: int64(3), object(9)
memory usage: 421.0+ KB


In [202]:
movie_resize.to_csv('./movie_resize.csv',index=False, encoding="utf-8-sig")

In [103]:
# 국적 데이터 변경
for idx, row in movie_resize.iterrows():
    if  row['국적'] not in ['미국', '한국', '일본', '영국', '프랑스','중국','독일']:
        movie_resize['국적'][idx] = '기타'

C:\Users\Playdata\AppData\Local\Temp\ipykernel_928\1179239548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_resize['국적'][idx] = '기타'


In [104]:
movie_resize['국적'].value_counts()

미국     1527
한국     1276
일본      409
기타      407
영국      187
프랑스     164
중국      118
독일       57
Name: 국적, dtype: int64

In [105]:
movie_resize.loc[(movie_resize['전국 관객수']>2500000)]

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분
순번,,,,,,,,,,,,
1,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화
2,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화
3,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화
4,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화
5,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...
326,정글북,존 파브로,월트디즈니컴퍼니코리아(주),2016-06-09,장편,미국,924,21457807134,2532051,어드벤처,12세관람가,일반영화
328,목격자,조규장,(주)넥스트엔터테인먼트월드(NEW),2018-08-15,장편,한국,1021,21732042597,2524086,스릴러,15세관람가,일반영화
330,마이펫의 이중생활,크리스 리노드,유니버설픽쳐스인터내셔널 코리아(유),2016-08-03,장편,미국,726,19379904655,2517805,애니메이션,전체관람가,일반영화


In [478]:
movie_resize.head(200)

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분
순번,,,,,,,,,,,,
1,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화
2,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화
3,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화
4,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화
5,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화
6,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,애니메이션,전체관람가,일반영화
7,아바타,제임스 카메론,주식회사 해리슨앤컴퍼니,2009-12-17,장편,미국,912,128447097523,13624328,SF,12세관람가,독립/예술영화
8,베테랑,류승완,(주)씨제이이엔엠,2015-08-05,장편,한국,1064,105024756250,13395400,액션,15세관람가,일반영화
10,도둑들,최동훈,(주)쇼박스,2012-07-25,장편,한국,1072,93665568500,12983330,액션,15세관람가,일반영화


In [106]:
movie_labels = pd.get_dummies(movie_resize,columns = ['영화형태','국적','장르','등급','영화구분'])

In [445]:
not_used = ['영화명', '감독', '배급사','개봉일','전국스크린수', '전국 매출액']

movie_DL = movie_labels.drop(not_used, axis=1)

In [446]:
독립 = movie_DL[['영화형태_단편', '영화형태_옴니버스', '영화형태_장편', '국적_기타',
       '국적_독일', '국적_미국', '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF',
       '장르_가족', '장르_공연', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마',
       '장르_멜로/로맨스', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)',
       '장르_성인물(에로)', '장르_스릴러', '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁',
       '장르_코미디', '장르_판타지', '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가',
       '영화구분_독립/예술영화', '영화구분_일반영화']] 
종속 = movie_DL[['전국 관객수']]

In [456]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target,test_target = train_test_split(독립,종속,test_size=0.2,random_state=42)

In [450]:
X = tf.keras.layers.Input(shape=[38])

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)
Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X,Y)
sgd = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.5)
model.compile(optimizer=sgd,loss='MAE')

In [454]:
model.fit(train_input,train_target,batch_size=128,epochs=100,validation_data=(test_input,test_target))

Epoch 1/100
26/26 [==============================] - 0s 5ms/step - loss: 478180.3750 - val_loss: 608502.2500
Epoch 2/100
26/26 [==============================] - 0s 4ms/step - loss: 477597.4375 - val_loss: 608581.6250
Epoch 3/100
26/26 [==============================] - 0s 4ms/step - loss: 479069.7500 - val_loss: 610731.6875
Epoch 4/100
26/26 [==============================] - 0s 4ms/step - loss: 476767.9375 - val_loss: 610883.4375
Epoch 5/100
26/26 [==============================] - 0s 4ms/step - loss: 477439.4688 - val_loss: 610672.1250
Epoch 6/100
26/26 [==============================] - 0s 4ms/step - loss: 477385.8438 - val_loss: 608809.8125
Epoch 7/100
26/26 [==============================] - 0s 4ms/step - loss: 478052.8750 - val_loss: 610752.6250
Epoch 8/100
26/26 [==============================] - 0s 5ms/step - loss: 479229.5000 - val_loss: 608024.3750
Epoch 9/100
26/26 [==============================] - 0s 5ms/step - loss: 478153.6250 - val_loss: 609468.3750
Epoch 10/100
26/26 

In [452]:
model.predict(train_input[:5])

1/1 [==============================] - 0s 86ms/step


array([[  4651.7954],
       [ 16076.152 ],
       [576998.75  ],
       [ 13903.599 ],
       [ 94707.97  ]], dtype=float32)

In [453]:
train_target[:5]

,전국 관객수
순번,
5892,3883
208,3427464
856,945185
4473,11393
1025,704834


In [41]:
print(train_input.shape,test_input.shape)

(3316, 40) (829, 40)


In [468]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_jobs=-1,random_state=42)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.2890287911188757 0.20024582001965738


In [461]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[1.17347526e-06 4.05207792e-04 3.66939076e-04 4.55881888e-04
 4.46449503e-04 5.32706160e-02 3.18710118e-03 1.97891430e-03
 6.83958953e-04 1.12706215e-03 1.43036545e-01 7.40227683e-03
 2.57075363e-03 5.63022949e-05 4.18446155e-03 3.30730429e-04
 1.39028255e-03 1.32896309e-02 1.96751916e-02 1.25597593e-03
 3.31956936e-03 1.89290308e-02 5.50058172e-02 1.37338406e-02
 0.00000000e+00 4.41524924e-03 3.86286757e-03 1.11969891e-01
 1.93298494e-02 9.89046406e-04 9.57742289e-03 3.02297552e-02
 1.01352787e-01 6.31023361e-02 6.44794336e-02 1.39508626e-02
 1.13180015e-01 1.17456771e-01]


C:\Users\Playdata\AppData\Local\Temp\ipykernel_928\577101422.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


In [465]:
rf = RandomForestRegressor(n_jobs=-1,random_state=42,oob_score=True)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.20039212245239368


C:\Users\Playdata\AppData\Local\Temp\ipykernel_928\1731553993.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


In [469]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.29142111831427087 0.19230944105708184


In [470]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[1.37878365e-08 2.53358417e-04 3.00711917e-04 1.05745098e-04
 6.35439785e-04 5.35724386e-02 2.63023725e-03 3.20864658e-04
 4.15500351e-04 1.26525748e-03 1.47752024e-01 5.58862193e-03
 2.32413235e-03 1.28171874e-09 5.50169756e-04 3.10518833e-04
 7.61859684e-04 1.18183878e-02 2.57649706e-02 7.52614109e-04
 1.08078765e-03 1.36524755e-02 4.38026339e-02 1.32100132e-02
 0.00000000e+00 2.03062570e-03 1.35899139e-03 1.02397788e-01
 1.64673416e-02 5.36362882e-05 9.41582309e-03 1.90208036e-02
 1.21508707e-01 6.70984283e-02 7.72275796e-02 7.97645343e-03
 1.34230524e-01 1.14344520e-01]


C:\Users\Playdata\AppData\Local\Temp\ipykernel_928\1418952811.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  et.fit(train_input,train_target)


In [472]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(random_state=42)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.26083979409261737 0.2076936757053515
